In [ ]:
# important libraries
import requests
import pandas as pd
import sqlite3
import os
# Fetch launches
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
launches = response.json()
df = pd.DataFrame(launches)

# Clean column names
df.columns = df.columns.str.strip()

# Fetch rockets and launchpads for mapping
rockets = pd.DataFrame(requests.get("https://api.spacexdata.com/v4/rockets").json())
launchpads = pd.DataFrame(requests.get("https://api.spacexdata.com/v4/launchpads").json())
rocket_map = dict(zip(rockets["id"], rockets["name"]))
launchpad_map = dict(zip(launchpads["id"], launchpads["name"]))
launchpad_loc = dict(zip(launchpads["id"], launchpads[["latitude", "longitude"]].values.tolist()))

# Select and clean data
df = df[["name", "date_utc", "success", "rocket", "launchpad"]]
df["rocket_name"] = df["rocket"].map(rocket_map)
df["launchpad_name"] = df["launchpad"].map(launchpad_map)
df["year"] = pd.to_datetime(df["date_utc"]).dt.year
df["month"] = pd.to_datetime(df["date_utc"]).dt.month
# Avoid future warning
df["success"] = df["success"].fillna(False)
df = df.infer_objects(copy=False)
# Save to CSV and SQLite
os.makedirs("data", exist_ok=True)
df.to_csv("data/spacex_cleaned.csv", index=False)
conn = sqlite3.connect("data/spacex.db")
df.to_sql("launches", conn, if_exists="replace", index=False)
conn.close()

          name                  date_utc  success                    rocket  \
0    FalconSat  2006-03-24T22:30:00.000Z    False  5e9d0d95eda69955f709d1eb   
1      DemoSat  2007-03-21T01:10:00.000Z    False  5e9d0d95eda69955f709d1eb   
2  Trailblazer  2008-08-03T03:34:00.000Z    False  5e9d0d95eda69955f709d1eb   
3       RatSat  2008-09-28T23:15:00.000Z     True  5e9d0d95eda69955f709d1eb   
4     RazakSat  2009-07-13T03:35:00.000Z     True  5e9d0d95eda69955f709d1eb   

                  launchpad rocket_name   launchpad_name  year  month  
0  5e9e4502f5090995de566f86    Falcon 1  Kwajalein Atoll  2006      3  
1  5e9e4502f5090995de566f86    Falcon 1  Kwajalein Atoll  2007      3  
2  5e9e4502f5090995de566f86    Falcon 1  Kwajalein Atoll  2008      8  
3  5e9e4502f5090995de566f86    Falcon 1  Kwajalein Atoll  2008      9  
4  5e9e4502f5090995de566f86    Falcon 1  Kwajalein Atoll  2009      7  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 9

C:\Users\KOVVO\AppData\Local\Temp\ipykernel_3592\1702166454.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["success"] = df["success"].fillna(False)
